In [ ]:
import os
import warnings
from os import getenv

import cv2
import nopdb
import numpy as np
import PIL
import torch
from matplotlib import pyplot as plt

from spoof.dataset.dataset import FaceDataset
from spoof.model.vit import ViT

warnings.filterwarnings("ignore")

#### Load and visualize an image returned from the ```FaceDataset```

In [ ]:
# Setup dataset to get a random image and read it
os.chdir(getenv("HOME") + "/mipt/spoof")
ds = FaceDataset("data/casia/test/annotations.csv")
img = ds[np.random.randint(len(ds))]["image"]

In [ ]:
# Define some functions to map the tensor back to an image
def inv_normalize(tensor):
    """Normalize an image tensor back to the 0-255 range."""
    tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min()) * (256 - 1e-5)
    return tensor


def inv_transform(tensor, normalize=True):
    """Convert a tensor to an image."""
    if normalize:
        tensor = inv_normalize(tensor)
    array = tensor.detach().cpu().numpy()
    array = array.transpose(1, 2, 0).astype(np.uint8)
    return PIL.Image.fromarray(array)

In [ ]:
inv_transform(img)

#### Load the model

In [ ]:
# Create model and let it load the weights in the constructor
model = ViT()

#### Capture attention weights

In [ ]:
# def predict(input):
#     """Run the model on an input and print the predicted classes with probabilities."""
#     with torch.no_grad():
#         return model.get_liveness_score(model(input))


def capture_attn_call(input, layer_idx):
    with nopdb.capture_call(
        model.extractor.encoder.layers[layer_idx].self_attention.forward
    ) as attn_call:
        with torch.no_grad():
            model(input)
    return attn_call


# Get the attention matrix for the last layer
attn_matrix = capture_attn_call(img, 11).locals["attn_output_weights"][0]
print(f"Attention matrix shape: {attn_matrix.shape}")

#### Visualize attention map 

In [ ]:
def get_attention_map(img, layer_idx, get_mask=False):
    """Get the attention map for an image."""
    attn_matrix = capture_attn_call(img, layer_idx).locals["attn_output_weights"]

    # Average the attention weights across all heads.
    attn_matrix = torch.mean(attn_matrix, dim=1)

    # To account for residual connections, we add an identity matrix to the
    # attention matrix and re-normalize the weights.
    residual_att = torch.eye(attn_matrix.size(1))
    aug_att_mat = attn_matrix + residual_att
    aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)

    # Recursively multiply the weight matrices
    joint_attentions = torch.zeros(aug_att_mat.size())
    joint_attentions[0] = aug_att_mat[0]

    for n in range(1, aug_att_mat.size(0)):
        joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n - 1])

    v = joint_attentions[-1]
    grid_size = int(np.sqrt(aug_att_mat.size(-1)))
    mask = v[0, 1:].reshape(grid_size, grid_size).detach().numpy()

    if get_mask:
        result = cv2.resize(mask / mask.max(), (img.shape[1], img.shape[2]))
    else:
        mask = cv2.resize(mask / mask.max(), (img.shape[1], img.shape[2]))[..., np.newaxis]
        # print(f"MASK SHAPE: {mask.shape}")
        # print(f"IMG TRANSPOSE SHAPE: {img.numpy().transpose(1, 2, 0).shape}")
        # print(f"IMG TRANSPOSE: {img.numpy().transpose(1, 2, 0)[0, :10, :1]}")
        # print(f"AFTER MASK: {mask * img.numpy().transpose(1, 2, 0)}")
        # print(f"AFTER UINT8: {(mask * img.numpy().transpose(1, 2, 0)).astype('uint8')}")

        result = mask * img.numpy().transpose(1, 2, 0)

    return result

In [ ]:
def show_attention(img, get_mask=False):
    # Plot the attention maps for each layer without mask
    fig, ax = plt.subplots(figsize=(16, 16))
    fig.suptitle(f"Attention Maps, mask={get_mask}", fontsize=12)
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        attn_map = get_attention_map(img, layer_idx=i, get_mask=get_mask)
        ax.set_title(f"Layer {i + 1}")
        ax.imshow(attn_map)
        ax.axis("off")
    plt.tight_layout()
    plt.show()
    plt.savefig(f"attn_map_mask={get_mask}.png")

#### Visualize without mask

In [ ]:
show_attention(img)

#### Visualize with mask

In [ ]:
# Plot the attention maps for each layer with mask
show_attention(img, get_mask=True)